In [13]:
import numpy as np
from numpy.linalg import inv
from numpy import linalg as LA


In [14]:
def l1decode_pd(x0, A, y, pdtol, pdmaxiter):
    # shape of A matrix
    N = x0.shape[0]  # column
    M = y.shape[0]  # row

    # backtracking parameters
    alpha = 0.01
    beta = 0.5

    # update parameter
    mu = 10

    gradf0 = np.vstack((np.zeros((N, 1)), np.ones((M, 1))))

    # initialization on primal variables x and u
    x = x0
    Ax = A @ x
    u = (0.95) * abs(y - Ax) + (0.10) * max(abs(y - Ax))

    # inequality constraints
    fu1 = Ax - y - u
    fu2 = -Ax + y - u

    # dual variables initialization
    lamu1 = -1. / fu1
    lamu2 = -1. / fu2

    # the first line of dual residual, but it's kind of strange
    # since we dont have equality constrains here, in other words
    # Atv does not exis
    Atv = A.T @ (lamu1 - lamu2)

    # initialize surrogate duality gap and tau
    sdg = -(fu1.T @ lamu1 + fu2.T @ lamu2)
    tau = mu * 2 * M / sdg  # multiply by 2 since we divide constrains into two parts
    # we have totally 2M inequality constrains

    # initialize rcent, rdual and resnorm
    # rcent is central residual
    # rdual is dual residual
    # resnorm is used to check if the rdual and rcent is small enough
    rcent = np.vstack((-lamu1 * fu1, -lamu2 * fu2)) - (1 / tau)
    rdual = gradf0 + np.vstack((Atv, -lamu1 - lamu2))
    resnorm = LA.norm(np.vstack((rdual, rcent)))

    pditer = 0
    done = (sdg < pdtol) | (pditer >= pdmaxiter)

    while (not done):

        pditer += 1

        w2 = -1 - 1 / tau * (1 / fu1 + 1 / fu2)

        sig1 = -lamu1 / fu1 - lamu2 / fu2
        sig2 = lamu1 / fu1 - lamu2 / fu2
        sigx = sig1 - sig2 ** 2 / sig1

        w1 = -1 / tau * (A.T @ (-1 / fu1 + 1 / fu2))

        # w1p and H11p are used to solve dx, conjugate gradients algorithm are
        # used in the origal code, but it has been modified
        w1p = w1 - A.T @ ((sig2 / sig1) * w2)
        # H11p = AtDiagA(AtA,sigx)


        sigx = sigx.squeeze()
        H11p = A.T @ (np.diag(sigx)) @ A;

        dx = np.linalg.solve(H11p, w1p)
        Adx = A @ dx

        # solve for primal variable
        du = (w2 - sig2 * Adx) / sig1
        # solve for dual variables
        dlamu1 = -(lamu1 / fu1) * (Adx - du) - lamu1 - (1 / tau) * 1 / fu1
        dlamu2 = (lamu2 / fu2) * (Adx + du) - lamu2 - (1 / tau) * 1 / fu2

        #
        Atdv = A.T @ (dlamu1 - dlamu2)

        # make sure that the step is feasible: keeps lamu1,lamu2 > 0, fu1,fu2 < 0
        indl = np.where(dlamu1 < 0);
        indu = np.where(dlamu2 < 0);
        #s = min([1, -lamu1[indl] / dlamu1[indl], -lamu2[indu] / dlamu2[indu]]);
        s = min([1, min(-lamu1[indl] / dlamu1[indl]), min(-lamu2[indu] / dlamu2[indu])]);
        indl = np.where((Adx - du) > 0);
        indu = np.where((-Adx - du) > 0);
        s = (0.99) * min([s, min(-fu1[indl] / (Adx[indl] - du[indl])), min(-fu2[indu] / (-Adx[indu] - du[indu]))])

        # backtrack
        suffdec = 0
        backiter = 0
        while (not suffdec):

            # primal variables update
            xp = x + s * dx;
            up = u + s * du;
            # dual variables update
            lamu1p = lamu1 + s * dlamu1;
            lamu2p = lamu2 + s * dlamu2;

            Axp = Ax + s * Adx;
            Atvp = Atv + s * Atdv;

            fu1p = Axp - y - up;
            fu2p = -Axp + y - up;

            rdp = gradf0 + np.vstack((Atvp, -lamu1p - lamu2p))
            rcp = np.vstack((-lamu1p * fu1p, -lamu2p * fu2p)) - (1 / tau)

            suffdec = (LA.norm(np.vstack((rdp, rcp))) <= (1 - alpha * s) * resnorm)
            s = beta * s;
            backiter = backiter + 1;
            if (backiter > 32):
                print('Stuck backtracking, returning last iterate.')
                xp = x
                break

        # next iteration
        x = xp;
        u = up;
        Ax = Axp;
        Atv = Atvp;
        lamu1 = lamu1p;
        lamu2 = lamu2p;
        fu1 = fu1p;
        fu2 = fu2p;

        # surrogate duality gap
        sdg = -(fu1.T @ lamu1 + fu2.T @ lamu2);
        tau = mu * 2 * M / sdg;
        rcent = np.vstack((-lamu1 * fu1, -lamu2 * fu2)) - (1 / tau)
        rdual = rdp;
        resnorm = LA.norm(np.vstack((rdual, rcent)))

        done = (sdg < pdtol) | (pditer >= pdmaxiter);

    return xp

In [15]:
# source length
N = 25

# codeword length
M = 4*N

# number of perturbations
T = round(0.2*M)

# coding matrix
G = np.random.randn(M,N)
# source word
x = np.random.randn(N,1)

# code word
c = np.dot(G,x)

# channel: perturb T randomly chosen entries
q = np.random.permutation(M)
y = c
y[q[0:T]] = np.random.randn(T,1)

# recover
x0 = inv(G.T@G)@G.T@y

#x0 = pinv(G)*y
#x0 = ones(256,1)


In [16]:
xp = l1decode_pd(x0, G, y, 1e-4, 30)

In [17]:
print(xp)

[[-0.18614281]
 [-0.37585412]
 [-0.08349852]
 [-0.54091633]
 [ 0.51629507]
 [-1.72420627]
 [-0.37768673]
 [ 0.89121659]
 [-0.34101819]
 [-0.80130331]
 [-0.24678955]
 [-1.34010971]
 [ 1.13643069]
 [-2.07866749]
 [-0.05619074]
 [-0.92340258]
 [ 0.74707276]
 [-1.09763079]
 [ 0.68913187]
 [-0.53694142]
 [-0.81727177]
 [-1.21309856]
 [ 0.40060537]
 [-0.24990324]
 [ 0.01037901]]


In [18]:
x

array([[-0.18614281],
       [-0.37585414],
       [-0.08349852],
       [-0.5409163 ],
       [ 0.51629506],
       [-1.72420628],
       [-0.37768676],
       [ 0.89121659],
       [-0.34101818],
       [-0.80130332],
       [-0.24678957],
       [-1.34010971],
       [ 1.13643072],
       [-2.0786675 ],
       [-0.05619071],
       [-0.9234026 ],
       [ 0.74707277],
       [-1.09763079],
       [ 0.68913191],
       [-0.53694141],
       [-0.81727177],
       [-1.21309856],
       [ 0.40060538],
       [-0.24990324],
       [ 0.01037901]])